# 用Jupyter玩MicroPython小車

## ESP12的電機板

- 電機電源（VM）：4.5V～36V，可單獨供電
- 控制電源（VIN）：4.5V～9V（10VMAX），可單獨供電
- 但我們這邊把VM, VIN短路，比較省事
- 其中，D1，D3（A電機）；D2，D4（B電機）

![電機板](image/motor_board.png)

### 供電

- 以行動電源供電即可

### 馬達

- TT減速馬達(48:1)

### 超音波測距模組

- HC-SR04+ : 給MicroPython 記得使用這一款，可以吃3.3V、5V的電，因為MicroPython是3.3V的GPIO

----

## 測試馬達

In [154]:
%websocketconnect --password 1234 ws://192.168.43.145:8266

** WebSocket connected **
Password: Ready.


In [155]:
# 定義
import machine
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM


In [156]:
#Setup PINS
led = machine.Pin(2, machine.Pin.OUT)

# for motor sheilf
motor_a1 = machine.Pin(D1, machine.Pin.OUT) #A-, speed
motor_a2 = machine.Pin(D3, machine.Pin.OUT) #A+, dir
motor_b1 = machine.Pin(D2, machine.Pin.OUT) #B-, speed
motor_b2 = machine.Pin(D4, machine.Pin.OUT) #B+, dir
FWD = 1 #high
REV = 0 #low


In [152]:
def car_fwd():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_rev():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(REV)

def car_stop():
    motor_a1.value(0)
    motor_a2.value(FWD)
    motor_b1.value(0)
    motor_b2.value(FWD)

def car_right():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_left():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(REV)


### 測試前進、後退、左轉、右轉

In [136]:
# 前進、後退
car_fwd()
time.sleep(3)
car_rev()
time.sleep(3)
car_stop()

.

In [119]:
car_fwd()

In [120]:
car_rev()

In [124]:
car_stop()

In [122]:
car_right()

In [123]:
car_left()

In [144]:
#全部動作跑一次
car_fwd()
time.sleep(2)
car_right()
time.sleep(2)
car_left()
time.sleep(2)
car_rev()
time.sleep(2)
car_stop()

..

----

## 超音波 test

- HC-SR04


In [147]:
%websocketconnect --password 1234 ws://192.168.43.230:8266

** WebSocket connected **
Password: Ready.


In [148]:
# 定義
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM


In [131]:
# 340m/s --> 340000mm/s --> 1mm需要2.94 us
# 340m/s --> 34000cm/s --> 1cm要29.4us --> 去回，所以除以(29.4*2)=58.8，但手冊是寫58us為1cm
# 所以如果目標是50cm內的量測距離 --> 50*58=2900us為最長時間
# timeout回傳0，很怪，應該給個大數字比較好，比如999

import time
from machine import Pin

def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58

while True:
    distance=ping(trigPin=D7,echoPin=D8)
    print('%s cm' % distance)
    time.sleep(1)

7.18966 cm
7.56896 cm
7.58621 cm
7.56896 cm
9.60345 cm
7.56896 cm
6.81034 cm
6.7931 cm
.7.58621 cm
7.56896 cm
7.56896 cm
9.60345 cm
7.56896 cm
9.62069 cm
7.43103 cm
.7.56896 cm
999 cm
999 cm
999 cm
999 cm
999 cm
999 cm
.999 cm
999 cm
11.7069 cm
14.5345 cm
14.5172 cm
14.5345 cm
.14.5172 cm
22.3276 cm
25.1379 cm
28.0 cm
28.0172 cm
34.9655 cm
40.2586 cm
.39.8793 cm
39.8793 cm
39.8965 cm
40.569 cm
40.0517 cm
40.6552 cm
.40.6552 cm
40.6552 cm


*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 48, in <module>
KeyboardInterrupt: 


## 前進 + 避障

- 先引入Ping函式
- 再開始前進
- 這邊很單純的碰到障礙，就右轉、或後退

In [149]:
import time
from machine import Pin

def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58


#distance=ping(trigPin=D7,echoPin=D8)

In [153]:
car_fwd()

while(True):
    distance=ping(trigPin=D7,echoPin=D8)
    if distance>10:
        car_fwd()
    elif distance<=15:
        car_right()
    elif distance<=10:
        car_stop()
        time.sleep(0.5)
    elif distance<=5:
        car_rev()
    time.sleep(0.1)

.........................

***OSError [遠端主機已強制關閉一個現存的連線。]



----

### 燒入MicroPython板子

- MicroPython一上電會執行 main.py 的程式，因此我們使用magic command "sendtofile" 燒入板子裡


In [170]:
%websocketconnect --password 1234 ws://192.168.43.230:8266

** WebSocket connected **
Password: Ready.


In [172]:
%sendtofile main.py

# 定義
import machine
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

#Setup PINS
led = machine.Pin(2, machine.Pin.OUT)
for i in range(5):
    led.value(not led.value())

# for motor sheilf
motor_a1 = machine.Pin(D1, machine.Pin.OUT) #A-, speed
motor_a2 = machine.Pin(D3, machine.Pin.OUT) #A+, dir
motor_b1 = machine.Pin(D2, machine.Pin.OUT) #B-, speed
motor_b2 = machine.Pin(D4, machine.Pin.OUT) #B+, dir
FWD = 1 #high
REV = 0 #low

def car_fwd():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_rev():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(REV)

def car_stop():
    motor_a1.value(0)
    motor_a2.value(FWD)
    motor_b1.value(0)
    motor_b2.value(FWD)

def car_right():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_left():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(REV)


def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58

# go~
car_fwd()

while(True):
    distance=ping(trigPin=D7,echoPin=D8)
    if distance>20:
        car_fwd()
    elif distance>10:
        car_right()
    elif distance<=10:
        car_rev()
    time.sleep(0.2)

Sent 109 lines (2320 bytes) to main.py.


----

### magic command 測試

In [69]:
%lsmagic

%capture [--quiet] [--QUIET] outputfilename
    records output to a file

%comment
    print this into output

%disconnect [--raw]
    disconnects from web/serial connection

%esptool [--port PORT] {erase,esp32,esp8266} [binfile]
    commands for flashing your esp-device

%fetchfile [--binary] [--print] [--quiet] [--QUIET]
                  sourcefilename [destinationfilename]
    fetch and save a file from the device

%lsmagic
    list magic commands

%mpy-cross [--set-exe SET_EXE] [pyfile]
    cross-compile a .py file to a .mpy file

%readbytes
    does serial.read_all()

%readbytes [--binary]
    does serial.read_all()

%rebootdevice
    reboots device

%sendtofile [--append] [--mkdir] [--binary] [--execute]
                   [--source [SOURCE]] [--quiet] [--QUIET]
                   [destinationfilename]
    send cell contents or file/direcectory to the device

%serialconnect [--raw] [--port PORT] [--baud BAUD] [--verbose]
    connects to a device over USB wire

%socketconnect [--

## 修改連接的AP

In [102]:
%serialconnect to --port=com13 --baud=115200

Connecting to --port=com13 --baud=115200 
Ready.


- 用magic comand的方式

In [98]:
%sendtofile ap.txt
my-ap-name
my-password

Sent 2 lines (18 bytes) to ap.txt.


- %%writefile這一個指令是指寫到電腦端

In [92]:
%%writefile ap.txt

my-ap-name
my-password

Writing ap.txt



### 寫檔的方式修改連結的AP

In [79]:
ap_config_fn = 'ap.txt'

In [80]:
AP_SSID = 'my-ap-name'
AP_PWD = 'my-password'
with open(ap_config_fn, 'w') as f:
    f.write(AP_SSID+'\n')
    f.write(AP_PWD+'\n')

In [99]:
ap_config = None

print('ap config here!')
with open(ap_config_fn, 'r') as f:
    ap_config = f.read()

print(ap_config)
ap_config = ap_config.split('\n')
AP_SSID = ap_config[0].strip()
AP_PWD = ap_config[1].strip()
print(ap_config)

ap config here!
malo-ap
0928380233
['malo-ap', '0928380233']
